# TopoMLP

# Abstract

TopoMLP is a transfromer based model, capable of multi-task learning - 3D Lane Detection, Traffic Element Detection, Topology prediction among lanes and Topology between lane and traffic element. It uses the state-of-the-art 3D Object detection model PETR. For traffic element detection, it uses a combination of Deformable DETR transformer model and YoloV8. It uses 2 Multi-Layer Perceptron layers for topology prediction(Lane-Lane and Lane-Traffic). This model acts as a baseline model which can be further improved to better predict lanes and topology.


# Architecture

## Lane Detection
The lane detector utilizes Bezier curves with 3D control points to represent each lane's centerline. This representation, combined with 3D position embeddings and transformer-based feature updating, allows for precise and smooth lane detection in multi-view images.

### Feature Extraction
* TopoMLP uses ResNet50 model as a backbone to generate feature maps from multi-view images, F ∈ R ^ V ×C×H×W , where V, C, H, and W represent the view number, channel, height, and width of the features

### 3D Position Embedding
* 3d position embeddings are encoded into the generated visual features to create position aware features. This embedding helps the model understand the spatial relationships within the 3D space. 

### Lane Anchor Points
* `NL` learnable 3D lane anchor points QL ∈ R^(NL×3) are initialized. These anchor points serve as the initial guesses for the positions of lane control points
* The anchor points' feature dimensions are projected from 3D to the number of channels `C` using a position encoding and a linear layer. This transformation is done to match the dimensions of the feature maps `C` to enable interaction.


### Transformer Decoder
* The transformer points are fed into a transformer decoder layer to update lane query features `QˆL`.

    QˆL = LaneDecoder(F , Linear(Q
L
))∈R
NL×C

### Output
* Two MLPs are used to predict the offset of control points(3d lane points modeled using Bezier curve) and classification scores for lane detection. The final control points are obtained by adding the offsets to the initial lane anchor points





## Traffic Element Detection

* Utilizes the Deformable Detr model to predict bounding boxes and classification scores. The Deformable DETR uses query embeddings to generate reference points(anchor points) on objects, this paper modifies the reference format to generate reference boxes(center points, height and width). The approach uses YoloV8 to generate object reference points, where each query represents a potential object. These query embeddings interact with object feature maps in the transformer decoder through multi-headed attention mechanism.

### Feature Generation
* Object features are extracted from the front view(front-center in case of openlane). These features are fed into YOLOV8 to generate object proposal points(anchor points), which are concatenated with a set of reference boxes to create randomized queries Rt. The generated boxes by YOLOv8 are encoded by sine-cosine embedding to generate query features,
which are concatenated with the randomized queries, denoted as QT
. The query features as well as the reference boxes are fed into the deformable decoder

### TrafficDecoder:
The TrafficDecoder is a stack of Deformable decoder layers.

QˆT = TrafficDecoder(F
0
, Q
T
, R
T
)∈R
NT ×C

The query features and reference boxes interact within the decoder to produce decode traffic features, which are then fed into 2 MLPs for classification of objects represented in the predicted bounding boxes and predict the object locations.







## TOPOLOGY REASONING

* Lane embeddings are created from the lane coordinates using an MLP, which are then integrated into the decoded lane features from PETR. This operatin is formed Nl times in two different directions. These two matrices are then concatenated to form a new feature matrix, which is fed into an MLP for binary classification.